<Header> This is is where we develop a baseline factor model that allows us to take certain factors and use them in multi-OLS regression to predict our excess return signals on a security level. These signals are absolute. </Header>

In [1]:
#First, import our packages for the database connection and dataframe access
import mysql.connector
import pandas as pd
import statsmodels.api as sm
import numpy as np
import warnings
from datetime import datetime, timedelta

In [2]:
#Ignore warnings and set max row display option
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 300)

In [3]:
# Connect to MySQL database
connection = mysql.connector.connect(
    host="ubctg.con7266gcvin.us-east-2.rds.amazonaws.com",
    user="admin",
    password="ubctgquant",
    database="ubctg"
)

Here, we pull monthly returns across the stock universe over 10 years from 2011 to 2021

In [4]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT * FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}'"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
universe_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
universe_df = pd.DataFrame(universe_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Generate the number of observations per security in our table. This way we can remove those that do not have a full dataset

In [5]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Define the start and end dates
start_date = '2017-12-01'
end_date = '2021-01-31'

# SQL query to retrieve data from the "Volatility" table between two dates
sql_query = f"SELECT COUNT(PERMNO), PERMNO FROM `Monthly Returns` mr WHERE Date BETWEEN '{start_date}' AND '{end_date}' GROUP BY PERMNO"

# Execute the SQL query
cursor.execute(sql_query)

# Fetch all rows from the result set
observationTable_data = cursor.fetchall()

# Convert fetched data into a pandas DataFrame
columns = [i[0] for i in cursor.description]  # Extract column names from cursor description

#create new df
observation_df = pd.DataFrame(observationTable_data, columns=columns)

#close cursor and db connection
cursor.close()

True

Here, we remove securities that do not have enough observations (in our case, we look for at least 120)

In [6]:
#Filter securities with at least 120 observations in the period
observation_df_filtered = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=38]

#Inner join our dataframes to only keep the securities we have data on
universe_df_filtered = pd.merge(universe_df, observation_df_filtered, on='PERMNO', how='inner')

We can now add a few factors to our model, in this case we will use GDP, CPI (inflation data), and the unemployment rate in each month. We then add it to our monthly returns dataframe for our OLS regression

In [7]:
#Introduce our factor data below as a CSV (pandas datareader not currently working for FRED api)
macro_factors = pd.read_csv("UBCTG Factor Model Example - Macro FRED Data.csv")

#These dates are at the beginning of month, so we will operate on our monthly return dataframe to convert our dates to the beginning of the month so that we can append on index
universe_df_filtered["date"] = pd.to_datetime(universe_df_filtered["date"]).dt.to_period('M').dt.to_timestamp()

#Convert macro factors date column to type datetime64 and drop the non-datetime64 column
macro_factors["date"] = pd.to_datetime(macro_factors["DATE"])
macro_factors = macro_factors.drop('DATE', axis=1)

#Inner-join macro factors dataframe with universe dataframe, using 'date' column as index
universe_df_with_external_factors = pd.merge(universe_df_filtered, macro_factors, on= 'date', how='inner')

#Ensure no errors in the returns column (there have been some instances where returns have taken on non-numeric values)
universe_df_with_external_factors["RET"] = pd.to_numeric(universe_df_with_external_factors["RET"], errors="coerce")

In [8]:
#Create a blank row at the end of our dataframe to accomodate shifted data
universe_df_with_external_factors_elongated = pd.concat([universe_df_with_external_factors, pd.DataFrame([[np.nan] * universe_df_with_external_factors.shape[1]], columns=universe_df_with_external_factors.columns)], ignore_index=True)

#Move into factor model
#Shift in -1 direction, rename to shifted return or similar
#Shift our returns data forward by one period. This way, we regress "t" factors to "t+1" returns, and our betas become forecasts
universe_df_with_external_factors_elongated['RET'] = universe_df_with_external_factors_elongated['RET'].shift(-1)

#We want to remove our minimum date so that we have an equal number of periods across securities. Here we find the minimum date and remove rows with that date from our dataframe
dtrmval = np.min(universe_df_with_external_factors_elongated.date)
dtrmvalrangemax = dtrmval + timedelta(days=1)

#Filter out our beginning date
universe_df_with_external_factors_filtered = universe_df_with_external_factors_elongated[(universe_df_with_external_factors_elongated["date"] >= dtrmvalrangemax)]
#colnames = universe_df_with_external_factors_filtered.columns[4:6]
#print(colnames)

#display(universe_df_with_external_factors_filtered.iloc[:,3:])

#Here, we select our required columns in proper order (see function documentation for more info) and get rid of extra columns that we will not use
universe_df_with_external_factors_filtered = universe_df_with_external_factors_filtered[["PERMNO","date","RET", "Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted","CPI (USACPALTT01CTGYM)","UNRATE"]]
display(universe_df_with_external_factors_filtered)

PERMNO       date       RET  \
1       10026.0 2018-01-01  0.007743   
2       10026.0 2018-02-01 -0.088191   
3       10026.0 2018-03-01 -0.029688   
4       10026.0 2018-04-01  0.019951   
5       10026.0 2018-05-01  0.006224   
...         ...        ...       ...   
234583  93436.0 2020-09-01  0.741452   
234584  93436.0 2020-10-01 -0.139087   
234585  93436.0 2020-11-01 -0.095499   
234586  93436.0 2020-12-01  0.462736   
234587  93436.0 2021-01-01  0.243252   

        Annualized Percent Change of GDP from Preceding Period, Seasonally Adjusted  \
1                                                2.590361                             
2                                                3.685589                             
3                                                1.852684                             
4                                                1.741385                             
5                                                1.782396                             
...                                                   ...                             
234583                                          10.259221                             
234584                                          -3.677010                             
234585                                          -4.401027                             
234586                                           3.579485                             
234587                                           8.270387                             

        CPI (USACPALTT01CTGYM)  UNRATE  
1                     2.042188     4.0  
2                     2.175117     4.1  
3                     2.321362     4.0  
4                     2.406737     4.0  
5                     2.725657     3.8  
...                        ...     ...  
234583                1.332004     7.8  
234584                1.130982     6.8  
234585                1.122155     6.7  
234586                1.323012     6.7  
234587                1.365916     6.4  

[228146 rows x 6 columns]

In [9]:
#We generate our list of unique tickers, this can be done in multiple different ways (in this case, using a previous df with distinct tickers)
#listofPERMNO = observation_df[pd.to_numeric(observation_df['COUNT(PERMNO)']) >=37]
#uniqueTickerList = listofPERMNO['PERMNO']
#print(uniqueTickerList)
#uniquetickerlist1 = universe_df_with_external_factors_filtered.iloc[:,0].unique()

#universe_df_with_external_factors_filtered.columns
#universe_df_with_external_factors_filtered.iloc[:,0]
#print(universe_df_with_external_factors_filtered.iloc[:,0].unique())


In [9]:
#Defining the factor model that takes a dataframe of the required columns (unique identifier, date/index column, returns columns, and a set of factors)
def olsfactormodel(df_attached, lookbackwindow:int):

    #Assign passed dataframe to new dataframe
    df = df_attached.copy(deep=True)
    
    #We generate our list of unique tickers using the column in the unique identifier position (0, or leftmost column)
    uniqueTickerList = df.iloc[:,0].unique()
    
    #Initialize large df to drop results of regression for each security at each regression date
    containerdf = pd.DataFrame()
                
    #Set our lookback window to 24 periods (months in this case). This means that we will run a regression for each period (after the first 24 months) using the previous 24 months as data
    LookBack_Window=lookbackwindow
    
    #We will now create a new set of columns for the Beta and P-value for each of our factors. We will do this by iterating through each factor and columns for the beta coefficients
    factorlist = list(df.columns[3:])
    print(len(factorlist))
    
    #For each factor, define an empty column to hold the corresponding coefficient
    for factor in factorlist: 
        df["Beta_" + factor] = 0
    
    #Initialize global parameter(s) regardless of factor count
    df["R_squared"] = 0
    df["Constant B0"] = 0

    #For each identifier (ticker), generate a dataframe from the broader dataframe that 
    for ticker in uniqueTickerList:
        
        #For each unique identifier (ticker), we create a dataframe with observations from that particular identifier
        ticker_specific_universe_df = df.loc[df.iloc[:,0] == ticker]
        
        #We then sort our date/index column in position 1 to ensure our date is ascending from the earliest available
        ticker_specific_universe_df = ticker_specific_universe_df.sort_values(by= ticker_specific_universe_df.columns[1])

        #For each lookback window span, train an OLS and collect the results
        for x in range(0, len(ticker_specific_universe_df)-LookBack_Window):
            
            # Define the independent variables (X) and dependent variable (Y). X's are defined by our factor columns and our returns are defined in our third left-most column (position 2)
            X = ticker_specific_universe_df[factorlist][x:x+LookBack_Window]
            Y = ticker_specific_universe_df[ticker_specific_universe_df.columns[2]][x:x+LookBack_Window]
    
            #Add a constant term to the independent variables, check impact
            X = sm.add_constant(X)
        
            #Fit the linear regression model
            model = sm.OLS(Y, X)
            results = model.fit()
            results.params
            
            #Place our regression coefficients into their appropriate columns
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], "Constant B0"] = results.params[0]
            ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], "R_squared"] = results.rsquared
    
            #We do the same dynamically using our list of factors and placing into the appropriate factor column
    
            #Define an index (starting position for inserting regression params)
            i = 3 + len(factorlist) - 1
            
            for index in range(len(factorlist)):
                #increase our indices by one for each factor
                index+=1
                i+=1
                
                ticker_specific_universe_df.loc[ticker_specific_universe_df.index[x+1+LookBack_Window], ticker_specific_universe_df.columns[i]] = results.params[index]
        
         
        #Add ticker dataframe to larger container dataframe
        containerdf = pd.concat([containerdf, ticker_specific_universe_df], ignore_index=True)   
    return containerdf

In [10]:
returnsdf = olsfactormodel(universe_df_with_external_factors_filtered,35)

3


In [11]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
display(returnsdf[(returnsdf != 0).all(1)])

PERMNO       date       RET  \
36      10026.0 2021-01-01  0.072598   
73      10028.0 2021-01-01  0.125541   
110     10032.0 2021-01-01  0.046848   
147     10044.0 2021-01-01 -0.051522   
184     10051.0 2021-01-01 -0.030851   
221     10065.0 2021-01-01  0.036571   
258     10066.0 2021-01-01       NaN   
295     10104.0 2021-01-01  0.120755   
332     10107.0 2021-01-01  0.039006   
369     10113.0 2021-01-01  0.054937   
406     10138.0 2021-01-01  0.061920   
443     10145.0 2020-12-01  0.241891   
444     10145.0 2021-01-01  0.043056   
481     10158.0 2021-01-01  0.172615   
518     10200.0 2021-01-01  0.010334   
555     10201.0 2021-01-01  0.019417   
592     10207.0 2021-01-01  0.050960   
629     10220.0 2021-01-01  0.059775   
666     10232.0 2020-12-01  0.129542   
667     10232.0 2021-01-01 -0.039539   
704     10239.0 2021-01-01 -0.040588   
741     10252.0 2021-01-01  0.086635   
778     10253.0 2021-01-01  0.162011   
815     10257.0 2021-01-01  0.021930   
852     10258.0 2021-01-01 -0.221333   
889     10294.0 2021-01-01  0.020800   
926     10308.0 2021-01-01  0.109888   
963     10318.0 2021-01-01  0.116790   
1000    10333.0 2021-01-01  0.332830   
1037    10355.0 2021-01-01  0.496296   
1074    10382.0 2021-01-01 -0.002069   
1111    10397.0 2021-01-01 -0.017004   
1148    10421.0 2021-01-01 -0.061030   
1185    10443.0 2021-01-01 -0.005241   
1222    10463.0 2021-01-01 -0.040956   
1259    10501.0 2021-01-01  0.072449   
1296    10514.0 2021-01-01  0.145636   
1333    10516.0 2021-01-01  0.012859   
1370    10517.0 2021-01-01 -0.002247   
1407    10530.0 2021-01-01 -0.011111   
1444    10547.0 2021-01-01  0.039529   
1481    10550.0 2021-01-01 -0.176768   
1518    10606.0 2021-01-01  0.038839   
1555    10629.0 2021-01-01  0.089468   
1592    10645.0 2021-01-01  0.374874   
1629    10649.0 2021-01-01  0.062907   
1666    10661.0 2021-01-01  0.055556   
1703    10693.0 2021-01-01  0.029660   
1740    10696.0 2021-01-01 -0.011460   
1777    10777.0 2021-01-01  0.087308   
1851    10781.0 2021-01-01  0.004963   
1888    10812.0 2021-01-01 -0.019480   
1925    10860.0 2021-01-01 -0.117917   
1962    10866.0 2021-01-01  0.335599   
1999    10874.0 2021-01-01  0.021436   
2036    10886.0 2021-01-01  0.049383   
2073    10890.0 2021-01-01  0.349794   
2110    10892.0 2021-01-01  0.177026   
2147    10901.0 2021-01-01 -0.100000   
2184    10909.0 2021-01-01  0.051577   
2221    10932.0 2021-01-01  0.113901   
2258    10933.0 2021-01-01  0.061068   
2295    10952.0 2020-12-01  0.276874   
2296    10952.0 2021-01-01  0.275819   
2333    10966.0 2021-01-01  0.252624   
2370    11006.0 2021-01-01  0.061868   
2407    11018.0 2021-01-01  0.161209   
2444    11043.0 2021-01-01  0.178241   
2481    11144.0 2021-01-01  0.025066   
2518    11153.0 2021-01-01       NaN   
2555    11154.0 2021-01-01 -0.037101   
2592    11174.0 2021-01-01  0.051100   
2629    11203.0 2020-12-01  0.107063   
2630    11203.0 2021-01-01  0.115080   
2667    11208.0 2021-01-01  0.111045   
2704    11264.0 2021-01-01  0.228784   
2741    11267.0 2021-01-01  0.189826   
2778    11275.0 2021-01-01  0.337778   
2815    11285.0 2021-01-01  0.083811   
2852    11292.0 2021-01-01 -0.044745   
2889    11293.0 2021-01-01  0.057632   
2926    11308.0 2021-01-01  0.062791   
2963    11343.0 2021-01-01 -0.033131   
3000    11348.0 2021-01-01  0.046820   
3037    11368.0 2020-11-01 -0.009379   
3038    11368.0 2020-12-01  0.140641   
3039    11368.0 2021-01-01  1.263636   
3076    11369.0 2021-01-01  0.118511   
3113    11370.0 2021-01-01  0.025293   
3150    11379.0 2021-01-01       NaN   
3187    11394.0 2021-01-01 -0.160643   
3224    11397.0 2021-01-01  0.152594   
3261    11403.0 2021-01-01  0.173087   
3298    11404.0 2021-01-01 -0.052197   
3335    11407.0 2021-01-01  0.186861   
3372    11441.0 2021-01-01  0.147727   
3409    11442.0 2021-01-01  0.058761   
3446    11481.0 2021-01-01  0.116436   
3483    11499.0 2021-01-01 -0.059459   
3520    1

In [13]:
list(range(0,10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]